#### Create a temporary directory to hold a local package.

In [1]:
%goflags --cover --covermode=set

%goflags=["--cover" "--covermode=set"]


In [2]:
!*TMP_PKG=$(mktemp -d /tmp/gonb_tests_gowork_XXXXXXXX) \
  && echo -n "${TMP_PKG}" > TMP_PKG.txt
!*cat TMP_PKG.txt

/tmp/gonb_tests_gowork_Xocdresp

#### Create dummy Go module

In [3]:
func Assert(err error) {
    if err != nil {
        panic(err)
    }
}

func Must[T any](v T, err error) T {
    Assert(err)
    return v
}

var TmpPkg string

func init_TmpPkg() {
    fPath := path.Join(os.Getenv("GONB_TMP_DIR"), "TMP_PKG.txt")
    TmpPkg = string(Must(os.ReadFile(fPath)))
}

%%
fmt.Printf("Temporary test package: %s\n", TmpPkg)

Assert(os.WriteFile(path.Join(TmpPkg, "go.mod"), []byte("module a.com/a/pkg\n\ngo 1.21\n"),0660))
Assert(os.WriteFile(path.Join(TmpPkg, "pkg.go"), []byte(
    "package pkg\n\nfunc A() string { return \"A\" }\n"),0660))

Temporary test package: /tmp/gonb_tests_gowork_Xocdresp


In [4]:
!*cat "$(cat TMP_PKG.txt)"/*

module a.com/a/pkg

go 1.21
package pkg

func A() string { return "A" }


#### Test creating `go.work` and using `%goworkfix`

In [5]:
!*rm -f go.work && go work init && go work use . "$(cat TMP_PKG.txt)"
%goworkfix

	- Added replace rule for module "a.com/a/pkg" to local directory "/tmp/gonb_tests_gowork_Xocdresp".


In [6]:
!*cat go.mod | sed 's|'"$(cat TMP_PKG.txt)"'|TMP_PKG|g;'

module gonb_bedb8330

go 1.23.5

replace a.com/a/pkg => TMP_PKG


#### Check for tracked directories

In [7]:
%track

List of files/directories being tracked: 
 
 /tmp/gonb_tests_gowork_Xocdresp

In [8]:
!*rm -f go.work
!*go mod edit -dropreplace=a.com/a/pkg
!*cat go.mod

module gonb_bedb8330

go 1.23.5


In [9]:
%untrack /tmp/gonb_tests_gowork_...
%track

	Untracked "/tmp/gonb_tests_gowork_..."


No files or directory being tracked yet

#### Clean up temporary directory

In [10]:
!*rm -r "$(cat TMP_PKG.txt)"